# Deep Learning

Deep Learnings has been introduced to NLP tasks in 2010s. One of the earliest approach is RNN and LSTM which work best for sequence like input. In this Notebook, we will experiment two different LSTM model on our IMDB dataset. Then, similar to Notebook 3, we will use pre-trained Word2Vec as input layer and see if it helps. Finally, we will use CNN which is a Deep Learning technique commonly used in Computer Vision.

In [39]:
%load_ext autoreload
%autoreload

from lib.dataset import download_tfds_imdb_as_text_tiny, download_tfds_imdb_as_text
from lib.deep_learning import run_lstm_pipeline, run_cnn_pipeline


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
dataset = download_tfds_imdb_as_text()
dataset_tiny = download_tfds_imdb_as_text_tiny()

# Basic LSTM

We will try two slightly different uses of LSTM in text classification

- BiLSTMLastStateClassification - We use the Bi-directional LSTM to encode a vector that represent the meaning of the whole sentence, then feed it to two layers of fully connected neural networks to make a prediction. This model is very different from what we have done in Notebook 1 - 3 in a sense that it takes "order" in to account (all previous experiments are bag-of-word). For example, it can differentiate this classic example `This movie is boring it is not good` and `This movie is good it is not boring`. Although bigrams can differentiate this particular example, in the real world there are many other examples that bigram is not enough and you need 3-gram, 4-gram. Increasing n-gram is not the solution because it introduced sparseness to feature space. That's why LSTM comes in. The drawback of this model is that if the sentence is very long, the last output state may encode very little information of the very first tokens of the sentence. 
<img src="tmp/last_state.png" width="800">

- BiLSTMPoolClassification - This is another use of LSTM. Instead of using the output of the last state of LSTM, now we use the LSTM output of every token in the sentence and, sum them up, and feed to two layers of FC layer. Since we sum up to output vector of each token, we go back to the bag-of-word approach again. However, it's not "very" bag-of-word. This approach is different from other bag-of-word style we used previously e.g. the Word2Vec bag-of-word in Notebook 2 - 3. In those models, we the word representing each tokens are static i.e. the vector for `good` are always same regarding where it presents in the sentence. However, in this architecture, the vector representing each token is contextualized It is aware of the word nearby because it's the output of LSTM. It can learn that vector for `good` can be different in different context. Moreover, since we use every LSTM output of each token, they also include the last state output as we used in above model. Although this vector is corresponding to the last token, it also represents the meaning of the whole sentence.

<img src="tmp/lstm_sum.png" width="500">

The  BiLSTMLastStateClassification is very similar to the TensorFlow official tutorial [here](https://www.tensorflow.org/tutorials/text/text_generation). However, we use the words as features to streamline with other experiments from Notebook 1 -4 while the tutorial use subwords. The model used in that tutorial has the F1 of 0.87.




In [3]:
run_lstm_pipeline(dataset, "BiLSTMLastStateClassification", embeddings_size=64, hidden_unit=64)


Epoch 1, Loss: 0.61, Accuracy: 0.65, Test Loss: 0.40, Test Accuracy: 0.83, Time: 342.28 s
Epoch 2, Loss: 0.29, Accuracy: 0.89, Test Loss: 0.30, Test Accuracy: 0.88, Time: 356.27 s
Epoch 3, Loss: 0.18, Accuracy: 0.94, Test Loss: 0.30, Test Accuracy: 0.88, Time: 358.60 s
Epoch 4, Loss: 0.13, Accuracy: 0.96, Test Loss: 0.31, Test Accuracy: 0.88, Time: 357.05 s
Epoch 5, Loss: 0.09, Accuracy: 0.97, Test Loss: 0.36, Test Accuracy: 0.86, Time: 354.26 s
Epoch 6, Loss: 0.06, Accuracy: 0.98, Test Loss: 0.36, Test Accuracy: 0.87, Time: 356.94 s
Epoch 7, Loss: 0.04, Accuracy: 0.99, Test Loss: 0.49, Test Accuracy: 0.87, Time: 349.87 s
Epoch 8, Loss: 0.03, Accuracy: 0.99, Test Loss: 0.48, Test Accuracy: 0.87, Time: 355.96 s
Epoch 9, Loss: 0.02, Accuracy: 1.00, Test Loss: 0.61, Test Accuracy: 0.86, Time: 354.42 s
Epoch 10, Loss: 0.02, Accuracy: 0.99, Test Loss: 0.71, Test Accuracy: 0.86, Time: 359.26 s


In [10]:
run_lstm_pipeline(dataset, "BiLSTMPoolClassification", embeddings_size=64, hidden_unit=64)


Epoch 1, Loss: 0.70, Accuracy: 0.69, Test Loss: 0.45, Test Accuracy: 0.81, Time: 376.92 s
Epoch 2, Loss: 0.41, Accuracy: 0.83, Test Loss: 0.37, Test Accuracy: 0.85, Time: 381.55 s
Epoch 3, Loss: 0.28, Accuracy: 0.90, Test Loss: 0.30, Test Accuracy: 0.88, Time: 380.45 s
Epoch 4, Loss: 0.22, Accuracy: 0.92, Test Loss: 0.28, Test Accuracy: 0.89, Time: 399.71 s
Epoch 5, Loss: 0.16, Accuracy: 0.94, Test Loss: 0.29, Test Accuracy: 0.89, Time: 372.07 s
Epoch 6, Loss: 0.14, Accuracy: 0.95, Test Loss: 0.32, Test Accuracy: 0.88, Time: 377.14 s
Epoch 7, Loss: 0.11, Accuracy: 0.96, Test Loss: 0.33, Test Accuracy: 0.88, Time: 370.48 s
Epoch 8, Loss: 0.07, Accuracy: 0.98, Test Loss: 0.40, Test Accuracy: 0.88, Time: 371.49 s
Epoch 9, Loss: 0.08, Accuracy: 0.97, Test Loss: 0.49, Test Accuracy: 0.88, Time: 381.28 s
Epoch 10, Loss: 0.04, Accuracy: 0.99, Test Loss: 0.58, Test Accuracy: 0.87, Time: 370.52 s


From this experiment, we can see that BiLSTMPoolClassification which uses all states output not just last state have to higher F1 score (0.88 vs 0.87). However,it's slightly slower to train. Note that these two models have similar number of parameters and layers. The only different is how to connect these layers.

# Basic LSTM with Pre-trained Word2Vec 

In this experiment, we will see if it helps if we initialize the embeddings with pre-trained Word2Vec. Similar to the Notebook 3, the idea is to see if it's useful to introduce the knowledge of the language to the model via pre-trained Word2Vec. Since the dimension of Word2Vec is 300, we have to change our embeddings layer size to be 300.

**Prerequisite**

Download [Google Word2Vec Model](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing) to this directory and run 

```
gunzip GoogleNews-vectors-negative300.bin.gz
```
If you already have those files or you don't want to save it in this directory, you can either change constant variable PRETRAINED_WV_MODEL_PATH  and PRETRAINED_GLOVE_MODEL_PATH or create symbolic link.
    
```
ln -s /path/to/your/word2vec ./GoogleNews-vectors-negative300.bin

```

In [5]:
# load pre-trained word embeddings from disk - take about 5 mins to run
import gensim 
PRETRAINED_WV_MODEL_PATH = "./GoogleNews-vectors-negative300.bin"
word2vec = gensim.models.KeyedVectors.load_word2vec_format(PRETRAINED_WV_MODEL_PATH, binary=True)


In [6]:
run_lstm_pipeline(dataset, "BiLSTMLastStateClassification", embeddings_size=300, hidden_unit=64, word2vec=word2vec)


Epoch 1, Loss: 0.65, Accuracy: 0.61, Test Loss: 0.55, Test Accuracy: 0.75, Time: 586.90 s
Epoch 2, Loss: 0.47, Accuracy: 0.79, Test Loss: 0.41, Test Accuracy: 0.83, Time: 615.34 s
Epoch 3, Loss: 0.34, Accuracy: 0.87, Test Loss: 0.35, Test Accuracy: 0.86, Time: 602.34 s
Epoch 4, Loss: 0.27, Accuracy: 0.90, Test Loss: 0.34, Test Accuracy: 0.87, Time: 608.78 s
Epoch 5, Loss: 0.21, Accuracy: 0.93, Test Loss: 0.35, Test Accuracy: 0.85, Time: 611.19 s
Epoch 6, Loss: 0.18, Accuracy: 0.94, Test Loss: 0.34, Test Accuracy: 0.87, Time: 608.22 s
Epoch 7, Loss: 0.14, Accuracy: 0.95, Test Loss: 0.36, Test Accuracy: 0.87, Time: 611.84 s
Epoch 8, Loss: 0.10, Accuracy: 0.97, Test Loss: 0.41, Test Accuracy: 0.87, Time: 610.79 s
Epoch 9, Loss: 0.09, Accuracy: 0.97, Test Loss: 0.43, Test Accuracy: 0.86, Time: 615.30 s
Epoch 10, Loss: 0.08, Accuracy: 0.98, Test Loss: 0.48, Test Accuracy: 0.85, Time: 602.85 s


In [11]:
run_lstm_pipeline(dataset, "BiLSTMLastStateClassification", embeddings_size=300, hidden_unit=64)


Epoch 1, Loss: 0.54, Accuracy: 0.71, Test Loss: 0.33, Test Accuracy: 0.86, Time: 605.13 s
Epoch 2, Loss: 0.23, Accuracy: 0.91, Test Loss: 0.34, Test Accuracy: 0.86, Time: 630.32 s
Epoch 3, Loss: 0.13, Accuracy: 0.96, Test Loss: 0.33, Test Accuracy: 0.88, Time: 635.41 s
Epoch 4, Loss: 0.08, Accuracy: 0.98, Test Loss: 0.36, Test Accuracy: 0.87, Time: 642.02 s
Epoch 5, Loss: 0.06, Accuracy: 0.98, Test Loss: 0.48, Test Accuracy: 0.87, Time: 629.13 s
Epoch 6, Loss: 0.03, Accuracy: 0.99, Test Loss: 0.46, Test Accuracy: 0.86, Time: 632.40 s
Epoch 7, Loss: 0.03, Accuracy: 0.99, Test Loss: 0.50, Test Accuracy: 0.86, Time: 637.20 s
Epoch 8, Loss: 0.02, Accuracy: 1.00, Test Loss: 0.66, Test Accuracy: 0.86, Time: 635.34 s
Epoch 9, Loss: 0.01, Accuracy: 1.00, Test Loss: 0.62, Test Accuracy: 0.85, Time: 656.13 s
Epoch 10, Loss: 0.01, Accuracy: 1.00, Test Loss: 0.75, Test Accuracy: 0.85, Time: 693.38 s


From this experiment, we have similar conclusion as in Notebook 3 that using pre-train Word2Vec does not help.

# Basic LSTM with Lingustic Features

In previous architecture, the word, for example, `good`, can have two different meaning depend on context. It can be adjective or noun. If it is adjective, it may be an indicator of positive sentiment. If it is noun, it may not be an indicator. However, the word embeddings will be the same regarding its meaning. In order to capture this different, some research papers use the linguistic features (part of speech and named entity) augmented with the word embeddings. This can be one-hot encoding (not trainable) or another embeddings (trainable). In this experiment, we will have two embeddings, one for word (dim=64) and one for part of speech (dim=10) and concatenate these vectors before pass to the LSTM as shown in the figure below.


<img src="tmp/lstm_pos.png" width="700">

In [41]:
run_lstm_pipeline(dataset, "BiLSTMPoolLinguisticClassification", embeddings_size=64, hidden_unit=64)

Epoch 1, Loss: 0.66, Accuracy: 0.69, Test Loss: 0.44, Test Accuracy: 0.84, Time: 351.60 s
Epoch 2, Loss: 0.41, Accuracy: 0.82, Test Loss: 0.39, Test Accuracy: 0.83, Time: 371.94 s
Epoch 3, Loss: 0.27, Accuracy: 0.89, Test Loss: 0.31, Test Accuracy: 0.88, Time: 372.50 s
Epoch 4, Loss: 0.34, Accuracy: 0.89, Test Loss: 0.29, Test Accuracy: 0.88, Time: 375.26 s
Epoch 5, Loss: 0.17, Accuracy: 0.94, Test Loss: 0.29, Test Accuracy: 0.89, Time: 377.27 s
Epoch 6, Loss: 0.15, Accuracy: 0.95, Test Loss: 0.29, Test Accuracy: 0.89, Time: 382.66 s
Epoch 7, Loss: 0.13, Accuracy: 0.95, Test Loss: 0.37, Test Accuracy: 0.85, Time: 375.88 s
Epoch 8, Loss: 0.10, Accuracy: 0.97, Test Loss: 0.37, Test Accuracy: 0.87, Time: 373.63 s
Epoch 9, Loss: 0.20, Accuracy: 0.94, Test Loss: 0.34, Test Accuracy: 0.88, Time: 373.66 s
Epoch 10, Loss: 0.07, Accuracy: 0.98, Test Loss: 0.35, Test Accuracy: 0.88, Time: 378.05 s


# Basic CNN

CNN is widely used in Computer Vision. However, recent NLP researchers also adopt CV for many different NLP tasks. The CNN is different from LSTM in the sense that it examines few tokens within window size (5 in our experiment) then move to the next window. Then it pools (taking average in our experiment) the output from each windows to the single vector. Intuitively, the CNN should be able to capture the local semantic within the window size better.

This [paper](https://arxiv.org/pdf/1702.01923.pdf) discuss the comparative study of CNN and RNN for NLP tasks. 

<img src="tmp/cnn.png" width="900">

In [8]:
run_cnn_pipeline(dataset, embeddings_size=64, hidden_unit=64, window=5)

Epoch 1, Loss: 0.69, Accuracy: 0.58, Test Loss: 0.69, Test Accuracy: 0.65, Time: 37.03 s
Epoch 2, Loss: 0.66, Accuracy: 0.74, Test Loss: 0.63, Test Accuracy: 0.77, Time: 35.41 s
Epoch 3, Loss: 0.59, Accuracy: 0.81, Test Loss: 0.56, Test Accuracy: 0.81, Time: 35.70 s
Epoch 4, Loss: 0.50, Accuracy: 0.85, Test Loss: 0.48, Test Accuracy: 0.84, Time: 35.35 s
Epoch 5, Loss: 0.43, Accuracy: 0.87, Test Loss: 0.43, Test Accuracy: 0.86, Time: 35.53 s
Epoch 6, Loss: 0.37, Accuracy: 0.89, Test Loss: 0.39, Test Accuracy: 0.87, Time: 36.00 s
Epoch 7, Loss: 0.32, Accuracy: 0.90, Test Loss: 0.36, Test Accuracy: 0.88, Time: 35.59 s
Epoch 8, Loss: 0.29, Accuracy: 0.91, Test Loss: 0.34, Test Accuracy: 0.88, Time: 35.61 s
Epoch 9, Loss: 0.26, Accuracy: 0.92, Test Loss: 0.32, Test Accuracy: 0.88, Time: 35.80 s
Epoch 10, Loss: 0.24, Accuracy: 0.93, Test Loss: 0.31, Test Accuracy: 0.89, Time: 35.58 s
Epoch 11, Loss: 0.22, Accuracy: 0.93, Test Loss: 0.30, Test Accuracy: 0.89, Time: 35.67 s
Epoch 12, Loss: 0.2

From this experiment, we can see that CNN has the F1 of 0.89. However, it has the fewest number of parameters.